In [1]:
from pandas import pandas as pd
#install with pip
from pandas import pandas as pd
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
from urllib.error import HTTPError

import datetime
import re
import sqlite3
import urllib
import csv
import datetime

In [ ]:
# Read in state summary report for District, filter for needed fields and assign headers
insp = pd.read_csv("ftp://dbprftp.state.fl.us/pub/llweb/5fdinspi.csv", 
                   usecols=[2, 4, 5, 6, 7, 8, 9, 12, 13, 14, 17, 18, 80, 81], encoding="ISO-8859-1")
insp.columns = ["county", "licnum", "sitename", "streetaddy", "cityaddy", "zip",
                "inspnum", "insptype", "inspdispos", "inspdate", "totalvio", "highvio", "licid", "visitid"]
# county_sought = 'Marion' # uncomment if particular county sought
# insp = insp[(insp.county == county_sought)] # uncomment if particular county sought
insp.sitename = insp.sitename.str.title() #titlecase sitename
insp.streetaddy = insp.streetaddy.str.title() #titlecase street adress
insp.cityaddy = insp.cityaddy.str.title() #titlecase city
insp = insp.applymap(lambda x: str(x).strip() if len(str(x).strip()) else None) #strip whitespace, replace empty vals
insp['visitid'] = insp['visitid'].apply(int) # so it can be filtered against df below

result = []
result_for_urls = result # for url list
result_for_tuple = result.append("NULL") # later into db, space for user-input datetime

# takes LicenseID and VisitID, passes it into the urls for detailed reports later
for index, rows in insp.iterrows():
    visitid = rows['visitid']
    licid = rows['licid']
    urls = "https://www.myfloridalicense.com/inspectionDetail.asp? \
        InspVisitID= %s &id= %s" % (visitid, licid)
    urls = urls.replace(' ', '')
    result.append(urls)
urlList = result
urlList.pop(0) # get rid of first "Null" from append above
print(urlList)

In [4]:
urlList = ['https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6632858&id=6605529', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6653464&id=4070634', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6602877&id=4070634', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6577708&id=2129302',]# a shorter try for a list
url = 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6632858&id=6605529'

html = urlopen(url)
soup = bs(html.read(), 'lxml')
inspurl = url
visitid = inspurl[inspurl.rfind("VisitID=") + 1:8]
details = soup.find_all('font', {'face': 'verdana'})[10:] # relevant text on pages in verdana

In [ ]:
site_name = details[0].text
high = int(details[24].text)
intermed = int(details[25].text)
basic = int(details[26].text)
total = high + intermed + basic
print(total)

In [20]:
site_name = []
visitid = []
high = []
intermed = []
basic = []
total = []
observed1 = []
observed2 = []
observed3 = []
observed4 = []
observed5 = []
observed6 = []
observed7 = []
observed8 = []
observed9 = []
observed10 = []
observed11 = []
observed12 = []

details_dict = {}
inspurl = url
visitid = inspurl[inspurl.rfind("VisitID=") + 1:8]
html = urlopen(url)
soup = bs(html.read(), 'lxml')
details = soup.find_all('font', {'face': 'verdana'})[10:] # relevant text on pages in verdana
result = []

site_name.append(details[0].text)
high.append(int(details[24].text))
intermed.append(int(details[25].text))
basic.append(int(details[26].text))
total.append(int(details[24].text) + (int(details[25].text) + (int(details[26].text))
if observed1.append(details[32].text):                               
    observed1.append(details[32].text)
else:
    print('No observations were recorded.')
if details[34].text:
    observed2.append(details[34].text)
else:
   break 
return result
    
for url in urlList:
    make_details_dict(url)

SyntaxError: invalid syntax (<ipython-input-20-bbc021134bb7>, line 33)